<table style="width: 100%;" id="nb-header">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 12pt;"><strong>Economic Models</strong>, Fall 2019<br>
            Dr. Eric Van Dusen</p></td></tr>
</table>

# Project 1: Modeling Macroeconomic Variables using the Cobb-Douglas Production Function

### Acknowledgements: ##
We would like to thank Professor Raymond Hawkins for his Economics 100B Problem Set that served as the basis for this assignment.

### Due Dates and Important Information:

- This project is in three parts. To get full credit for the checkpoint, you must pass all public autograder tests and write at least one word for the first two parts of the project. The checkpoint is due on Monday, October 21, 2019 at 11:59pm.
- The whole project (all three parts) will be due 2 Mondays from now, on October 28, 2019 at 11:59pm. You are allowed to amend parts 1 and 2 in this submission as well.
- **Do not change any of the given variable or function names as this would cause autograder problems. Make sure to name your columns and tables exactly as the questions ask you to.**

---

The goal of this project is to gain experience completing the following key steps in the data science pipeline:

1. Downloading and converting raw data files into a format Python can interact with
2. Identifying and visualizing overall trends in the data using a process called Exploratory Data Analysis (EDA)
3. Using the data to complete a problem of prediction

We hope that at the end of this project, you will see how the skills you have learned in Data 8 and this class can prepare you for dealing with real world datasets, and how you can use them to answer questions about the economy or the world.

**The question you will be answering today is the following:**

> How can we apply the Cobb-Douglas Production Function to understand the different ways countries produce output or GDP?

## Question 1: Retrieving the data and simplifying the problem

In almost all cases in industry, you will have to find the dataset that can answer the question you are trying to solve. However, as we are introducing you to this process, we will be providing the link to such a dataset.

We will be using "Penn World Table" (PWT), a dataset that catalogues information on relative levels of income, output, inputs and productivity for 182 countries between 1950 and 2017. It was compiled by the University of Groningen.

Your task:
1. Go to: https://www.rug.nl/ggdc/productivity/pwt/ and download the Excel datafile.
2. Open the Excel file, select the "Data" sheet, then File > Save As > Select. Type in a file name and **change the file format to Comma Separated Values (.csv)**.
3. Upload this .csv file to the same folder as this Notebook. You can access the Datahub file explorer by clicking the Jupyter logo on the top-left of the notebook and navigating to econ-fa19 > wk07. Uploading is accomplished by dragging and dropping the file into the file explorer window and clicking the blue button labeled "upload".
4. Edit the line ```file_name``` such that it matches the name of the .csv file that you just uploaded.
5. Run the code cell below.

In [ ]:
from utils import *
import pandas as pd
from datascience import *
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.linear_model import LinearRegression
import otter
grader = otter.Notebook()
%matplotlib inline
plt.style.use("seaborn-muted")

In [ ]:
file_name = ...

### Do not change the line below ###
data = to_table(file_name)

In [ ]:
grader.check("q1_0")

If you pass the autograder check above, then you have correctly read in the data file. Nice! You have converted an online dataset into a table that you can explore and manipulate in Python! Now, this dataset is saved under the table called ```data```. Look at the first 5 lines below:

In [ ]:
data.show(5)

Notice that there are a lot of ```-1``` values. This is representative of data that is unavailable for that variable, for a given country-year.

To get an idea of the dataset's geographic scope, let us find out what countries are included, and if they are spelt in interesting ways. This will be helpful for us later on in our analysis.

**Question 1.1:**
Create a two-column table called <code>all_countries</code>. Its first column will be called <code>Country</code> and the second <code>Earliest Year</code>. It should contain all of the countries that appear in the <code>data</code> table sorted in alphabetical order with the earliest year they appear in the dataset where the <code>cgdpe</code> column is not <code>-1</code>. Assign that table to the variable <code>all_countries</code> and display its first 10 rows.

In [ ]:
all_countries = ...
...

In [ ]:
grader.check("q1_1")

Take a look at ```data``` again. Notice that it has a lot of columns. We won't need most of them. As we will be using the Cobb-Douglas Production Function, think about what variables are needed in the equation, and which ones are already present in the table. This is an important part of the data science process: understanding the dataset that you are using. Most real-world datasets provide documentation listing the definitions and equations behind each variable. 

PWT provides this and has identified three variables that will be helpful to us:
1. cn $\Rightarrow$ Capital Stock in millions of USD
2. cgdpe $\Rightarrow$ Expenditure-side Real GDP in millions of USD
3. emp $\Rightarrow$ Number of Persons employed in millions

**An important note: for the sake of simplicity, we will be assuming that nations exhibit constant returns to scale.**

**Question 1.2(a):**
In the text cell below do the following:
1. Rewrite the original Cobb-Douglas function in LaTeX, under the assumption that are no constant returns to scale. On a separate line, write the restriction implied by constant returns to scale. <br />
2. Give a brief explanation of what each variable in the Cobb-Douglas function quantifies (i.e. K measures the amount of ...)

_Type your answer here, replacing this text._

**Question 1.2(b):**
Assign the variable <code>missing_variables</code> to an array containing the Cobb-Douglas function variables that are missing from the dataset.

In [ ]:
missing_variables = ...

In [ ]:
grader.check("q1_2b")

**Question 1.3:**
Remove all columns from <code>data</code> except for <code>cn</code>, <code>cgdpe</code>, <code>emp</code>, <code>country</code> and <code>year</code>. Ensure that <code>country</code> and <code>year</code> are the two left-most columns respectively. Call the new table <code>cleaned_data</code> and display its first five rows. Rename the <code>cn</code> column to <code>Capital Stock</code>, <code>cgdpe</code> to <code>Real GDP</code> and <code>emp</code> to <code>Labor Force</code>.
</div>

In [ ]:
cleaned_data = ...

In [ ]:
grader.check("q1_3")

Our goal will be to predict what $\alpha$ and $A$ are for each of the countries that we will be examining. From these, we will be able to explore how output is produced in each of these countries. The question is, how can we use the Cobb-Douglas Production function to solve for the missing variables? An easy way would be to take the natural log of the equation, making it linear, providing us with ways to quantify $\alpha$ and $A$.

**Question 1.4:**
In the cell below, using LaTeX, take the natural log of the Cobb-Douglas Production Function and rewrite it as a function of one variable. Show all of your work. Full credit will not be given if you just display the final simplified equation without showing any work. <br />
<i> Hint: $\ln (Y) - \ln (L) = \text{...}$ </i>

_Type your answer here, replacing this text._

## Question 2: Exploring the Data

Whenever we are dealing with a large dataset like PWT, it is a good idea to see how the variables interact with each other. A common method, especially when dealing with economic data, is to generate a time series. This is a plot of some economic variable on the y-axis and time on the x-axis. We are going to do just that for our important Cobb-Douglas variables for different countries.

**Question 2.1:**
Create a table of GDP, Capital Stock and Labor for the USA from 1990 to 2017. Do the same for China and two other countries of your choice. Use the middle cell to check if your countries of choice exist in the dataset. Make sure to use the country name exactly as it appears in the data table. Place all of this data in one table called <code>comparison_data</code>. Display the first 10 rows of <code>comparison_data</code>. Additionally, set the <code>country_array</code> variable to an array of the countries you selected in alphabetical order. <br />
<i>Hint: Look at the <a href="https://d8a-88.github.io/econ-fa19/python-reference.html"> Python Reference</a> for a table function you can use.</i>

In [ ]:
### Run this cell to see a table of all the countries in the dataset.###
### Select two countries and proceed to the lower code cell. ###
all_countries.show()

In [ ]:
# country_array should be in alphabetical order
country_array = make_array("United States", "China", ...)

comparison_data = ...

In [ ]:
grader.check("q2_1")

**Note:** If you ever need to refer to a list of the countries you selected in your code, do **not** use ```country_array```. When you place the data in ```comparison_data```, Python will automatically re-order the countries. Using ```country_array``` will cause a mis-match between your rows of data in ```comparison_data``` and the country it is actually from.

**Question 2.2:**
To help us in later questions, fill in the blanks in the <code>country_table_plotter</code> function below. Its inputs will be a table of the form <code>comparison_data</code> and the names of the two columns plotted, <code>columnX</code> and <code>columnY</code>. The <code>country_table_plotter</code> function will plot <code>columnX</code> versus <code>columnY</code> using data from <code>data_table</code> for all the countries contained in <code>data_table</code>. <br />
<i> Hint: Look at the <a href="https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.plot.html"> Matplotlib Plotting Reference</a> for ideas of what to place in the blanks. </i>

In [ ]:
def country_table_plotter(data_table, columnX, columnY):
    
    ### Your code here ###
    for country in ...:
        plt.plot(..., ..., label = country, linewidth = 1)
    
    ### Do not change the code below ###
    plt.legend()
    plt.xlabel(columnX)
    plt.ylabel(f"{columnY} (Logarithmic Scale)")
    plt.yscale("log")

**Question 2.3:**
Produce a plot of time and capital stock for the countries in your table. In the text cell below your plot, identify differences between the countries and discuss what surprised you.

In [ ]:
...

- United States:

- China:

- Your Country:

- Your Country:

**Question 2.4:**
Produce a similar plot, but this time of time and labor for the countries in your table. In the text cell below your plot, identify differences between the countries and discuss what surprised you.

In [ ]:
...

- United States:

- China:

- Your Country:

- Your Country:

**Question 2.5:**
Create a plot of time and GDP for the countries in your table. In the text cell below your plot, using your knowledge of the Cobb-Douglas Production Function, identify differences between the countries and discuss these in relation to your findings about each of the country's levels of capital stock and labor. Also note about how these have changed over time, if at all.

In [ ]:
...

- United States:

- China:

- Your Country:

- Your Country:

### Checkpoint: Complete Questions 1 and 2 by Monday, October 21 at 11:59pm.

To turn in this project, go to File > Download As, select Notebook and upload this .ipynb file to Gradescope under Project 1 Checkpoint - Notebook.

---

## Question 3: Prediction and Comparison

We are now going to provide numerical backing to your discussions of differences between the nations. We will predict values for $\alpha$ and $A$ for each of the 4 countries that we are examining. To do this, we will revisit the equation that you derived in question 1.4. 

**Question 3.1:**
A key step in the original paper by Cobb and Douglas was that they converted the data into an index. This is important because the data is measured in different units. Do this for each country and each of the variables in <code>comparison_data</code>. Let the 2011 value for each variable in each country be the base year, i.e. 100. Place this in a new table called <code>indexed_data</code> together with <code>country</code> and <code>year</code> columns. Display the first few rows.

Hint: The formula for calculating an index is as follows: $$\frac{Q_\text{current year}}{Q_\text{base year}} \cdot 100, \text{ for some variable } Q$$

In [ ]:
base_values = ...
countries = ...
indexed_Ks = make_array()
indexed_Ls = make_array()
indexed_Ys = make_array()

for country in countries:
    ...
    
indexed_data = Table().with_columns("country", ...,
                                    "year", ...,
                                    "Indexed K", ...,
                                    "Indexed L", ...,
                                    "Indexed Y", ...
                                   )

indexed_data.show(5)

In [ ]:
grader.check("q3_1")

**Question 3.2(a):**
Using the equation you derived in part 1 and the <code>indexed_data</code> table, calculate the two log ratios that you need to perform linear regression. Place them in the table <code>log_ratios</code> with <code>country</code> and <code>year</code> as the two leftmost columns respectively. Display the first 10 rows and name the columns appropriately. The <code>log_ratios</code> table should have 4 columns.
</div>

In [ ]:
log_ratios = ...

In [ ]:
grader.check("q3_2a")

**Question 3.2(b):**
Using the <code>country_table_scatter</code> function, plot the log ratios for each country from the <code>log_ratios</code> table below.
</div>

In [ ]:
### DO NOT EDIT THE CODE IN THIS CELL, just run it! ###

def country_table_scatter(data_table, columnX, columnY):
    # First getting a list of all the countries in data_table
    country_list = data_table.group("country").column("country")
    
    # For each country, creating a plot of columnX vs. columnY
    for country in country_list:
        curr_data_table = data_table.where("country", country)
        curr_data_table.scatter(columnX, columnY)
        plt.title(country)

In [ ]:
...

### Extra Credit 1

What do you notice about the scatter plot for the US? What could be a possible reason for this?

_Type your answer here, replacing this text._

To help you compare, the code below will plot all of the scatter plots on the same axis. Remember, the axes are in terms of logarithms. Thus, even small differences in slope or intercept would yield big changes.

In [ ]:
### Do not change the code below ###
all_scatter(log_ratios, "ln(K/L)", "ln(Y/L)")

Now, we want to make a linear approximation of the curves above. How will we do this? Through linear regression.

We will be using Scikit-Learn's (Sk-learn) linear regression function to get the $\alpha$ and $A$ values for each country. Here is the link to the documentation of the <a href = "https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html"> function</a>: 

Let us break down how this function works by taking a look at the examples section of the documentation.
``` 
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
```
The ```LinearRegression()``` function creates a LinearRegression object that has many different functions that we will be using, mainly ```.fit()``` and attributes like ```.coef_``` and ```.intercept_```. The ```.fit(X, y)``` function fits a line using the ```X``` and ```y``` arrays passed into it.

```
reg.coef_
array([1., 2.])
reg.intercept_ 
3.0000...
reg.predict(np.array([[3, 5]]))
array([16.])
```

After we have fit a line to the data, we can then get the corresponding attributes that we need: the slope coefficient and intercept. Note: the above example uses the equation:

$$
y = ax_0 + bx_1 + c
$$

Its data is in 3 dimensions, $x_0$, $x_1$ and $y$. Thus, ```reg.coef_``` returns a two element array, as there are two slope coefficients for x_0 and x_1. ```reg.intercept_``` still returns the intercept of the fitted line.

Our data is in 2 dimensions, x and y. Thus, the function will attempt to fit the following line for each country:

$$
y = a + bx
$$

The function will find the values of $a$ and $b$ that minimize the squared errors at each value of $x$. You don't need to know the specifics of this just yet. Your task will be to figure out what arrays should be passed in as $y$ and $x$. Furthermore, think about the relationship between $\alpha$ and $A$ with regards to $a$ and $b$ from this linear regression fit.

**Question 3.3(a):**
Using Sklearn's Linear Regression function, fit the data taken from each country. Think about what should be passed into the <code>fit()</code> function. If you are confused about this, take a look at the sklearn <a href ="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.fit"> documentation</a> or the example above.        

</div>

In [ ]:
### Create arrays of the data we will need from each country. ###
### Think about which variable should be on each axis. ###

china_x = ...
china_y = ...
us_x = ...
us_y = ...
country1_x = ...
country1_y = ...
country2_x = ...
country2_y = ...


### Do not change the code below ###

model_china = LinearRegression().fit(china_x.reshape(-1, 1), china_y.reshape(-1, 1))
model_us = LinearRegression().fit(us_x.reshape(-1, 1), us_y.reshape(-1, 1))
model_country1 = LinearRegression().fit(country1_x.reshape(-1, 1), country1_y.reshape(-1, 1))
model_country2 = LinearRegression().fit(country2_x.reshape(-1, 1), country2_y.reshape(-1, 1))

In [ ]:
grader.check("q3_3a")

**Question 3.3(b):**
Now that we have fit the data of each country, we can then retrieve the slope and intercept of each fit. Using the equation you derived in question 1.3., fill in the blanks in the print statements below such that they display the $\alpha$ and $A$ values for each country. Note that you will need to transform at least one of the variables.


In [ ]:
print(f"China alpha value: {...}")
print(f"United States alpha value: {...}")
print(f"... alpha value: {...}")
print(f"... alpha value: {...}")

In [ ]:
print(f"China A value: {...}")
print(f"United States A value: {...}")
print(f"... A value: {...}")
print(f"... A value: {...}")

**Question 3.4:**
With reference to the $\alpha$ and $A$ values for each of the countries you have examined, do they indicate about that country's ability to produce output as measured through GDP? Compare and contrast how each country allocates capital and labor when producing output. How about the role of technology or research and development? 3-4 sentences should suffice.

_Type your answer here, replacing this text._

### Extra Credit 2
A bonus point for each for the following: <br />
- If you select the country with the lowest $\alpha$ value among all the projects submitted. <br />
- If you select the country with the highest $A$ value among all the projects submitted. <br />


# Congratulations! You've completed the project!

# Submission

To turn in this project, go to File > Download As, select Notebook and upload this .ipynb file to Gradescope under Project 1 Final Submission - Notebook. Then, run the cell below to generate a PDF of this assignment. It will save that file to the same folder that the notebook is in. Download that pdf and upload it to Gradescope under Project 1 Final Submission - PDF.

In [ ]:
grader.export("proj01.ipynb")